Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [5]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [6]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [7]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [8]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [9]:
seed=19

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Problem 1-a plain logistic regression with L2 regularization

In [24]:
train_subset = 10000
beta = 0.1

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels], seed=seed))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + beta * tf.nn.l2_loss(weights)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


num_steps = 801

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Loss at step 0: 322.060852
Training accuracy: 7.1%
Validation accuracy: 9.8%
Loss at step 100: 1.000029
Training accuracy: 81.8%
Validation accuracy: 80.4%
Loss at step 200: 0.988567
Training accuracy: 81.8%
Validation accuracy: 80.5%
Loss at step 300: 0.987988
Training accuracy: 81.9%
Validation accuracy: 80.5%
Loss at step 400: 0.987820
Training accuracy: 81.9%
Validation accuracy: 80.5%
Loss at step 500: 0.987764
Training accuracy: 81.9%
Validation accuracy: 80.5%
Loss at step 600: 0.987745
Training accuracy: 81.9%
Validation accuracy: 80.5%
Loss at step 700: 0.987738
Training accuracy: 81.9%
Validation accuracy: 80.5%
Loss at step 800: 0.987736
Training accuracy: 81.9%
Validation accuracy: 80.5%
Test accuracy: 87.6%


### Problem 1 - b Neural Net with L2 regularization

In [34]:
batch_size = 128
beta = 0.01

n_hidden_nodes = [1024]
n_all_nodes = [image_size * image_size, *n_hidden_nodes, num_labels]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = [None]*(len(n_hidden_nodes) + 1)
  biases = [None]*(len(n_hidden_nodes) + 1)
  train_net_layer_out = [None]*(len(n_hidden_nodes) + 1)
  valid_net_layer_out = [None]*(len(n_hidden_nodes) + 1)
  test_net_layer_out = [None]*(len(n_hidden_nodes) + 1)
  
  for i in range(len(n_hidden_nodes) + 1):
    print(i, n_all_nodes[i], n_all_nodes[i+1])
    weights[i] = tf.Variable(tf.truncated_normal([n_all_nodes[i], n_all_nodes[i+1]], seed=seed))
    biases[i] = tf.Variable(tf.zeros([n_all_nodes[i+1]]))
    if i == 0:
      train_net_layer_out[0] = tf.nn.relu(tf.matmul(tf_train_dataset, weights[0]) + biases[0])
      valid_net_layer_out[0] = tf.nn.relu(tf.matmul(tf_valid_dataset, weights[0]) + biases[0])
      test_net_layer_out[0] = tf.nn.relu(tf.matmul(tf_test_dataset, weights[0]) + biases[0])
    elif i < len(n_hidden_nodes):
      train_net_layer_out[i] = tf.nn.relu(tf.matmul(train_net_layer_out[i-1], weights[i]) + biases[i])
      valid_net_layer_out[i] = tf.nn.relu(tf.matmul(valid_net_layer_out[i-1], weights[i]) + biases[i])
      test_net_layer_out[i] = tf.nn.relu(tf.matmul(test_net_layer_out[i-1], weights[i]) + biases[i])
    else:
      train_net_layer_out[i] = tf.matmul(train_net_layer_out[i-1], weights[i]) + biases[i]
      valid_net_layer_out[i] = tf.matmul(valid_net_layer_out[i-1], weights[i]) + biases[i]
      test_net_layer_out[i] = tf.matmul(test_net_layer_out[i-1], weights[i]) + biases[i]
  
  # Training computation.
  logits = train_net_layer_out[-1]
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + beta * sum([tf.nn.l2_loss(x) for x in weights])
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(valid_net_layer_out[-1])
  test_prediction = tf.nn.softmax(test_net_layer_out[-1])
    
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions, w = session.run(
      [optimizer, loss, train_prediction, weights], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

0 784 1024
1 1024 10
Initialized
Minibatch loss at step 0: 3515.713135
Minibatch accuracy: 7.8%
Validation accuracy: 22.5%
Minibatch loss at step 500: 21.215614
Minibatch accuracy: 87.5%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 0.939213
Minibatch accuracy: 81.2%
Validation accuracy: 83.3%
Minibatch loss at step 1500: 0.594387
Minibatch accuracy: 86.7%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 0.609362
Minibatch accuracy: 89.8%
Validation accuracy: 83.2%
Minibatch loss at step 2500: 0.716117
Minibatch accuracy: 83.6%
Validation accuracy: 83.6%
Minibatch loss at step 3000: 0.764459
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Minibatch loss at step 3500: 0.766863
Minibatch accuracy: 81.2%
Validation accuracy: 83.7%
Minibatch loss at step 4000: 0.669616
Minibatch accuracy: 86.7%
Validation accuracy: 84.0%
Minibatch loss at step 4500: 0.688470
Minibatch accuracy: 85.2%
Validation accuracy: 83.3%
Minibatch loss at step 5000: 0.698025
Minibatch accuracy: 

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [35]:
batch_size = 128
beta = 0.01

n_hidden_nodes = [1024]
n_all_nodes = [image_size * image_size, *n_hidden_nodes, num_labels]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = [None]*(len(n_hidden_nodes) + 1)
  biases = [None]*(len(n_hidden_nodes) + 1)
  train_net_layer_out = [None]*(len(n_hidden_nodes) + 1)
  valid_net_layer_out = [None]*(len(n_hidden_nodes) + 1)
  test_net_layer_out = [None]*(len(n_hidden_nodes) + 1)
  
  for i in range(len(n_hidden_nodes) + 1):
    print(i, n_all_nodes[i], n_all_nodes[i+1])
    weights[i] = tf.Variable(tf.truncated_normal([n_all_nodes[i], n_all_nodes[i+1]], seed=seed))
    biases[i] = tf.Variable(tf.zeros([n_all_nodes[i+1]]))
    if i == 0:
      train_net_layer_out[0] = tf.nn.relu(tf.matmul(tf_train_dataset, weights[0]) + biases[0])
      valid_net_layer_out[0] = tf.nn.relu(tf.matmul(tf_valid_dataset, weights[0]) + biases[0])
      test_net_layer_out[0] = tf.nn.relu(tf.matmul(tf_test_dataset, weights[0]) + biases[0])
    elif i < len(n_hidden_nodes):
      train_net_layer_out[i] = tf.nn.relu(tf.matmul(train_net_layer_out[i-1], weights[i]) + biases[i])
      valid_net_layer_out[i] = tf.nn.relu(tf.matmul(valid_net_layer_out[i-1], weights[i]) + biases[i])
      test_net_layer_out[i] = tf.nn.relu(tf.matmul(test_net_layer_out[i-1], weights[i]) + biases[i])
    else:
      train_net_layer_out[i] = tf.matmul(train_net_layer_out[i-1], weights[i]) + biases[i]
      valid_net_layer_out[i] = tf.matmul(valid_net_layer_out[i-1], weights[i]) + biases[i]
      test_net_layer_out[i] = tf.matmul(test_net_layer_out[i-1], weights[i]) + biases[i]
  
  # Training computation.
  logits = train_net_layer_out[-1]
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + beta * sum([tf.nn.l2_loss(x) for x in weights])
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(valid_net_layer_out[-1])
  test_prediction = tf.nn.softmax(test_net_layer_out[-1])
    
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    _step = step % 5 # limit the range of batch
    offset = (_step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions, w = session.run(
      [optimizer, loss, train_prediction, weights], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

0 784 1024
1 1024 10
Initialized
Minibatch loss at step 0: 3515.713135
Minibatch accuracy: 7.8%
Validation accuracy: 22.5%
Minibatch loss at step 500: 21.105795
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 1000: 0.546957
Minibatch accuracy: 99.2%
Validation accuracy: 79.7%
Minibatch loss at step 1500: 0.377130
Minibatch accuracy: 99.2%
Validation accuracy: 79.7%
Minibatch loss at step 2000: 0.362677
Minibatch accuracy: 99.2%
Validation accuracy: 79.7%
Minibatch loss at step 2500: 0.355031
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Minibatch loss at step 3000: 0.350144
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Minibatch loss at step 3500: 0.347217
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Minibatch loss at step 4000: 0.345192
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Minibatch loss at step 4500: 0.343439
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Minibatch loss at step 5000: 0.342372
Minibatch accu

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [85]:
batch_size = 128
beta = 0.01
dropout_keep_prop=0.8

n_hidden_nodes = [1024]
n_all_nodes = [image_size * image_size, *n_hidden_nodes, num_labels]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = [None]*(len(n_hidden_nodes) + 1)
  biases = [None]*(len(n_hidden_nodes) + 1)
  train_net_layer_out = [None]*(len(n_hidden_nodes) + 1)
  valid_net_layer_out = [None]*(len(n_hidden_nodes) + 1)
  test_net_layer_out = [None]*(len(n_hidden_nodes) + 1)
  
  for i in range(len(n_hidden_nodes) + 1):
    print(i, n_all_nodes[i], n_all_nodes[i+1])
    weights[i] = tf.Variable(tf.truncated_normal([n_all_nodes[i], n_all_nodes[i+1]], seed=seed))
    biases[i] = tf.Variable(tf.zeros([n_all_nodes[i+1]]))

    train_input = None
    valid_input = None
    test_input = None
    need_activation_func = None

    if i == 0:
      train_input, valid_input, test_input = tf_train_dataset, tf_valid_dataset, tf_test_dataset
      need_activation_func = True
    else:
      train_input, valid_input, test_input = train_net_layer_out[i-1], valid_net_layer_out[i-1], test_net_layer_out[i-1]
      if i < len(n_hidden_nodes):
        need_activation_func = True
      else:
        need_activation_func = False
    
    train_net_layer_out[i] = tf.matmul(train_input, tf.nn.dropout(weights[i], dropout_keep_prop, seed=seed)) + biases[i]
    valid_net_layer_out[i] = tf.matmul(valid_input, weights[i]) + biases[i]
    test_net_layer_out[i] = tf.matmul(test_input, weights[i]) + biases[i]
    
    if need_activation_func:
        train_net_layer_out[i] = tf.nn.relu(train_net_layer_out[i])
        valid_net_layer_out[i] = tf.nn.relu(valid_net_layer_out[i])
        test_net_layer_out[i] = tf.nn.relu(test_net_layer_out[i])
  
  # Training computation.
  logits = train_net_layer_out[-1]
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + beta * sum([tf.nn.l2_loss(x) for x in weights])
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(valid_net_layer_out[-1])
  test_prediction = tf.nn.softmax(test_net_layer_out[-1])
    
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    _step = step % 5 # limit the range of batch
    offset = (_step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions, w = session.run(
      [optimizer, loss, train_prediction, weights], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

0 784 1024
1 1024 10
Initialized
Minibatch loss at step 0: 3665.082520
Minibatch accuracy: 6.2%
Validation accuracy: 32.2%
Minibatch loss at step 500: 21.478188
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 1000: 0.606874
Minibatch accuracy: 98.4%
Validation accuracy: 79.4%
Minibatch loss at step 1500: 0.408612
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 0.394385
Minibatch accuracy: 99.2%
Validation accuracy: 79.3%
Minibatch loss at step 2500: 0.399304
Minibatch accuracy: 99.2%
Validation accuracy: 79.5%
Minibatch loss at step 3000: 0.382977
Minibatch accuracy: 99.2%
Validation accuracy: 79.7%
Minibatch loss at step 3500: 0.377700
Minibatch accuracy: 99.2%
Validation accuracy: 79.1%
Minibatch loss at step 4000: 0.370098
Minibatch accuracy: 98.4%
Validation accuracy: 79.9%
Minibatch loss at step 4500: 0.388414
Minibatch accuracy: 99.2%
Validation accuracy: 79.9%
Minibatch loss at step 5000: 0.372819
Minibatch accuracy

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [86]:
batch_size = 256
beta = 1e-5
dropout_keep_prop=0.5

n_hidden_nodes = [1024, 768, 256]
n_all_nodes = [image_size * image_size, *n_hidden_nodes, num_labels]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0, trainable=False)
  learning_rate = tf.train.exponential_decay(0.1, global_step, 100000, 0.96, staircase=True)
  
  # Variables.
  weights = [None]*(len(n_hidden_nodes) + 1)
  biases = [None]*(len(n_hidden_nodes) + 1)
  train_net_layer_out = [None]*(len(n_hidden_nodes) + 1)
  valid_net_layer_out = [None]*(len(n_hidden_nodes) + 1)
  test_net_layer_out = [None]*(len(n_hidden_nodes) + 1)
  
  for i in range(len(n_hidden_nodes) + 1):
    print(i, n_all_nodes[i], n_all_nodes[i+1])
    weights[i] = tf.Variable(tf.truncated_normal([n_all_nodes[i], n_all_nodes[i+1]], stddev=tf.sqrt(2./(image_size * image_size)), seed=seed))
    biases[i] = tf.Variable(tf.zeros([n_all_nodes[i+1]]))
    
    train_input = None
    valid_input = None
    test_input = None
    need_activation_func = None

    if i == 0:
      train_input, valid_input, test_input = tf_train_dataset, tf_valid_dataset, tf_test_dataset
      need_activation_func = True
    else:
      train_input, valid_input, test_input = train_net_layer_out[i-1], valid_net_layer_out[i-1], test_net_layer_out[i-1]
      if i < len(n_hidden_nodes):
        need_activation_func = True
      else:
        need_activation_func = False
    
    train_net_layer_out[i] = tf.matmul(train_input, tf.nn.dropout(weights[i], dropout_keep_prop, seed=seed)) + biases[i]
    valid_net_layer_out[i] = tf.matmul(valid_input, weights[i]) + biases[i]
    test_net_layer_out[i] = tf.matmul(test_input, weights[i]) + biases[i]
    
    if need_activation_func:
        train_net_layer_out[i] = tf.nn.relu(train_net_layer_out[i])
        valid_net_layer_out[i] = tf.nn.relu(valid_net_layer_out[i])
        test_net_layer_out[i] = tf.nn.relu(test_net_layer_out[i])

    
  
  # Training computation.
  logits = train_net_layer_out[-1]
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + beta * sum([tf.nn.l2_loss(x) for x in weights])
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(valid_net_layer_out[-1])
  test_prediction = tf.nn.softmax(test_net_layer_out[-1])
    
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

0 784 1024
1 1024 768
2 768 256
3 256 10
Initialized
Minibatch loss at step 0: 2.778881
Minibatch accuracy: 8.6%
Validation accuracy: 16.0%
Minibatch loss at step 100: 1.218712
Minibatch accuracy: 57.4%
Validation accuracy: 70.8%
Minibatch loss at step 200: 0.917404
Minibatch accuracy: 70.3%
Validation accuracy: 79.8%
Minibatch loss at step 300: 0.871234
Minibatch accuracy: 76.2%
Validation accuracy: 80.2%
Minibatch loss at step 400: 0.703575
Minibatch accuracy: 77.7%
Validation accuracy: 80.0%
Minibatch loss at step 500: 0.795217
Minibatch accuracy: 77.0%
Validation accuracy: 81.1%
Minibatch loss at step 600: 0.585033
Minibatch accuracy: 81.6%
Validation accuracy: 82.0%
Minibatch loss at step 700: 0.631099
Minibatch accuracy: 82.4%
Validation accuracy: 81.5%
Minibatch loss at step 800: 0.775660
Minibatch accuracy: 75.8%
Validation accuracy: 82.7%
Minibatch loss at step 900: 0.730208
Minibatch accuracy: 80.5%
Validation accuracy: 82.4%
Minibatch loss at step 1000: 0.570520
Minibatch ac

Validation accuracy: 86.4%
Minibatch loss at step 9000: 0.495378
Minibatch accuracy: 84.8%
Validation accuracy: 86.3%
Minibatch loss at step 9100: 0.457594
Minibatch accuracy: 86.3%
Validation accuracy: 86.5%
Minibatch loss at step 9200: 0.435519
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 9300: 0.487220
Minibatch accuracy: 84.8%
Validation accuracy: 86.8%
Minibatch loss at step 9400: 0.404519
Minibatch accuracy: 88.7%
Validation accuracy: 86.5%
Minibatch loss at step 9500: 0.329353
Minibatch accuracy: 89.8%
Validation accuracy: 86.6%
Minibatch loss at step 9600: 0.340611
Minibatch accuracy: 89.5%
Validation accuracy: 86.7%
Minibatch loss at step 9700: 0.582872
Minibatch accuracy: 81.6%
Validation accuracy: 86.9%
Minibatch loss at step 9800: 0.523619
Minibatch accuracy: 85.5%
Validation accuracy: 86.8%
Minibatch loss at step 9900: 0.349286
Minibatch accuracy: 90.2%
Validation accuracy: 86.7%
Minibatch loss at step 10000: 0.445790
Minibatch accuracy: 87.1